In [ ]:




# submit process

# Write Submission CSV
sub = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
sub = sub[['customer_id']]

In [ ]:


df.article_id.value_counts()#%%

In [ ]:
sub['customer_id_2'] = sub['customer_id'].apply(lambda x :  int( x[-16:],16))


sub = sub.merge(preds.rename({'customer_id':'customer_id_2'},axis=1), on='customer_id_2',how='left').fillna('')

del sub['customer_id_2']


sub.prediction

sub.prediction = sub.prediction + top12

sub.prediction


sub.prediction = sub.prediction.str.strip()
sub.prediction = sub.prediction.str[:131]

sub['prediction'].values[0]

sub.to_csv(f'submission.csv',index=False)
sub.head()

In [15]:



from src import data

In [31]:
import importlib
importlib.reload(data)

<module 'src.data' from 'C:\\Users\\DATA\\PycharmProjects\\kaggle\\H&MRecommendation\\src\\data.py'>

In [32]:
import os

os.getcwd()

'C:\\Users\\DATA\\PycharmProjects\\kaggle\\H&MRecommendation'

In [33]:
dataloader = data.DataLoader()

In [34]:
train = dataloader.load('train')

In [37]:
group_data = train[['customer_id','article_id','sales_channel_id']].groupby(['article_id','customer_id']).count().reset_index()


In [41]:
pivot_data = group_data.pivot(columns='article_id',index='customer_id',values='sales_channel_id')


IndexError: index 688600994 is out of bounds for axis 0 with size 688470939

,article_id,customer_id,sales_channel_id
0,0108775015,000f7535bdc611ad136a9f04746d6b1431f50a7f60fbbe...,3
1,0108775015,001ae5408a043f64bccd32beffe2730151414cbdf18a6e...,2
2,0108775015,001ba9e81e13ce12a2585d9ebde923fe74429e9e12ea59...,1
3,0108775015,0022a721371d5949d174ecba60346d89a9d6c08c0fba4f...,2
4,0108775015,002323971cbd38fad4512d5114676e5e17eb262db02320...,1
...,...,...,...
27306434,0956217002,be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...,1
27306435,0956217002,c89698e2626f3d98d8496e673c89ece5a8ee92914ee92d...,1
27306436,0956217002,cedd533807afaef98ca626a771d8c14986de84bdb43743...,1
27306437,0956217002,d51d1c7b07110ba2571d22849c1fdb7e61d7cbc1d58ada...,1


In [ ]:
pivot_data.fillna(0,inplace=True)

In [42]:
import time
import numpy as np
import pandas as pd
import multiprocessing as mp
from multiprocessing import Pool
from functools import partial

In [43]:

dfu = dataloader.load('dfu')

In [45]:
dfi = dataloader.load('dfi')

In [46]:
ALL_USERS = dfu['customer_id'].unique().tolist()

In [49]:
ALL_ITEMS = dfi['article_id'].unique().tolist()

In [50]:
user_to_customer_map = {user_id: customer_id for user_id, customer_id in enumerate(ALL_USERS)}

In [52]:
customer_to_user_map = {customer_id : user_id for user_id , customer_id in enumerate(ALL_USERS)}

In [53]:
item_to_article_map = {item_id : article_id for item_id , article_id in enumerate(ALL_ITEMS)}

In [65]:
article_to_item_map = {article_id : item_id for item_id, article_id in enumerate(ALL_ITEMS)}



In [66]:
del dfu , dfi


In [67]:
df = train.copy()

In [68]:
df['user_id'] = df['customer_id'].map(customer_to_user_map)

0                 2
1                 2
2                 7
3                 7
4                 7
             ...   
31788319    1371691
31788320    1371691
31788321    1371721
31788322    1371747
31788323    1371960
Name: user_id, Length: 31788324, dtype: int64

In [71]:
df['item_id'] = df['article_id'].map(article_to_item_map)


In [87]:
# BUILD MODEL

N_SIMILAR_USERS =30

MINIMUM_PURCHASES = 3

START_DATE = '2020-08-21'

DROP_PURCHASE_ITEMS =False

DROP_USER_FROM_HIS_NEIGHBORHOOD = False

TEST_RUN = False

TEST_SIZE = 1000



In [74]:
def flatten(l):
    """ Flatten a list of lists"""
    return [item for sublist in l for item in sublist]

In [76]:
def compare_vectors(v1, v2):
    """
    Compare lists of purchased product for two given users
    v1 stands for the "vector representation for user 1"  , which is a list of the purchases of u1

    Returns:
        A value between 0 and 1 (similarity)

    :param v1:
    :param v2:
    :return:
    """
    intersection = len(set(v1) & set(v2))
    denominator = np.sqrt(len(v1) * len(v2))

    return intersection / denominator

In [79]:
def get_similar_users(u , v , dfh):
    """
    Get the N_SIMILAR_USERS most similar users to the given one with their similarity score

    :param u: the user_id
    :param v:  the "vector" representation of the user (list of item_id)
    :param dfh: the "history of transactions" dataframe
    :return: tuple of lists ([similar_user_id] , [similarity scores])
    """


    similar_users = dfh.apply(lambda v_other : compare_vectors(v, v_other)).sort_values(ascending=False).head(N_SIMILAR_USERS+1)

    if DROP_USER_FROM_HIS_NEIGHBORHOOD:
        similar_users = similar_users[similar_users.index != u]

    return similar_users.index.tolist(), similar_users.tolist()


def get_items(u, v , dfh):
    """ Get the recommend items for a given users

    It will:
        1) Get similar users for the given user
        2) Obtain all the items those users purchased
        3) Rank them using similarity scores of the user that purchased them
        4) Return the 12 best ranked

    Arguments:
        u: the user_id,
        v: the "vector" representation of the user ( list of item_id)
        dfh : the " history of transactions" dataframe

    Returns:
        list of item_id of lenth at most 12
    """


    global i, n

    users, scores = get_similar_users(u, v , dfh)

    df_nn = pd.DataFrame({'user':users, 'score': scores})

    df_nn['items'] = df_nn.apply(lambda row: dfh.loc[row.user],axis=1)

    df_nn['weighted_items'] = df_nn.apply(lambda row: [(item, row.score) for item in row['items']], axis=1)

    recs = pd.DataFrame(flatten(df_nn['weighted_items'].tolist()),\
                        columns=['item','score']).groupby('item')['score'].sum().sort_values(ascending=False)


    if DROP_PURCHASE_ITEMS:
        recs = recs[~recs.index.isin(v)]
    # Keep the first 12 and get the item_ids
    i += 1
    if i % 200 == 0:
        pid = mp.current_process().pid
        print(f"[PID {pid:>2d}] Finished {i:3d} / {n:5d} - {i/n*100:3.0f}%"  )
    return recs.head(12).index.tolist()

In [81]:
def get_items_chunk(user_ids: np.array, dfh:pd.DataFrame):

    """Call get_item for a list of user_ids



    :param use_ids: list of user_id
    :param dfh: "history of transactions" dataframe
    :return: pd.Series with index user_id and list of item_id (recommendation) as value
    """
    global i, n
    i = 0

    n = len(user_ids)
    pid = mp.current_process().pid

    print(f"[PID {pid:>2d}] Started working with {n:5d} users")

    df_user_vectors = pd.DataFrame(dfh.loc[user_ids]).reset_index()
    df_user_vectors['recs'] = df_user_vectors.apply(lambda row: get_items(row.user_id,row.item_id,dfh),axis=1)
    return df_user_vectors.set_index('user_id')['recs']

SyntaxError: invalid syntax (Temp/ipykernel_2108/4015999658.py, line 25)

In [109]:


np.array(users, mp.cpu_count())

16

In [ ]:
def get_recommendation(users: list , dfh :pd.DataFrame):


    """
    Obtained recommendation for the users using transaction dfh in a parallelized manner

    Call get_items_chunk in a  "smart" multiprocessing fashion

    Arguments:
        users: lists of user_id
        dfh : the "history of transactions" dataframe
    :return
        pd.DataFrame with index user_id and list of item_id (recommendation) as value



    :param users:
    :param dfh:
    :return:
    """
    time_start = time.time()

    #Split into approximately evenly sized chunks
    # We will send just one batch to each CPU
    user_chunks = np.array_split(users, mp.cpu_count())


    f = partial(get_items_chunk,dfh=dfh)
    with Pool(mp.cpu_count()) as p:
        res = p.map(f, user_chunks)

    df_rec = pd.DataFrame(pd.concat(res))

    elapsed = (time.time() - time_start) / 60
    print(f"Finished get_recommendation({len(users)}). It took {elapsed:5.2f} mins ")
    return df_rec

    #%%


In [88]:
def uucf(df , start_date =START_DATE):

    """
    Entry poin for the UUCF model

    Receicve the original transactions_train.csv and a start_date and gets UUCF recommendations

    the model will not cover the full list of users, but just a subset of them.

    It will provid recommendations for users with at least MINIMUM_PURCHASES after start_date

    It might return less than 12 recs per user.

    An ad-hoc function for filling these gaps should be used downstream

    (See fill functionallity right below)

    Arguments:
        df : The raw dataframe from transaction_train.csv
        start_data : a date

    Returns:
        a submission-like pd.DataFrame with columns [customer_id, prediction]
        'prediction' is a list and not a string though



    :param df:
    :param start_data:
    :return:
    """

    df_small = df[df['t_dat'] > start_date]
    print(f"Kept data from {start_date} on. Total rows: {len(df_small)}")

    # H stands for "Transaction history"
    # dfh is a series of user_id => list of item_id (the list of purchases in order)
    dfh = df_small.groupby("user_id")['item_id'].apply(lambda items: list(set(items)))
    dfh = dfh[dfh.str.len() >= MINIMUM_PURCHASES]

    if TEST_RUN:
        print("WARNING : TEST_RUN is True . It will be a toy execution")
        dfh = dfh.head(TEST_SIZE)
    users = dfh.index.tolist()
    n_users = len(users)
    print(f"Total users in the time frame with at least {MINIMUM_PURCHASES}: {n_users}")


    df_rec = get_recommendation(users, dfh)
    df_rec['customer_id'] = df_rec.index.map(user_to_customer_map)
    df_rec['prediction'] = df_rec['recs'].map(lambda l : [item_to_article_map[i] for i in l])

    # Submission ready dataframe
    df_rec.reset_index(drop=True)[['customer_id','prediction']]
    return df_rec

In [91]:
df_small = df[df['t_dat'] > '2018-09-20']

In [93]:
dfh =df_small.groupby('user_id')['item_id'].apply(lambda  items: list(set(items)))

,t_dat,customer_id,article_id,price,sales_channel_id,user_id,item_id
48399,2018-09-21,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0583558001,0.067780,2,1,19333
48400,2018-09-21,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0639677008,0.025407,2,1,33748
48401,2018-09-21,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0640244003,0.033881,2,1,33991
48402,2018-09-21,0006d37aaf7dd84f9bbc02f6cadcb74fd72ebf370bdc5f...,0685687004,0.016932,2,168,46305
48403,2018-09-21,0006d37aaf7dd84f9bbc02f6cadcb74fd72ebf370bdc5f...,0591334004,0.010153,2,168,20909
...,...,...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,0929511001,0.059305,2,1371691,104961
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,0891322004,0.042356,2,1371691,100629
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,0918325001,0.043203,1,1371721,104053
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,0833459002,0.006763,1,1371747,88521


In [96]:
dfh[dfh.str.lower]

user_id
0          [16003, 65667, 100484, 16023, 38172, 50724, 83...
1          [22656, 25857, 42626, 47747, 19333, 22664, 582...
2          [91841, 59458, 60259, 78503, 67274, 93585, 401...
3                                             [64525, 61175]
4          [2181, 20517, 77915, 104812, 43442, 101367, 51...
                                 ...                        
1371975    [64256, 58501, 59144, 58376, 48776, 59147, 560...
1371976    [29698, 69137, 7192, 52766, 51748, 90151, 4768...
1371977    [1157, 53893, 35721, 78601, 88977, 47892, 1653...
1371978           [89864, 99149, 85872, 95506, 56694, 81751]
1371979                                              [81594]
Name: item_id, Length: 1361243, dtype: object

user_id
0          [16003, 65667, 100484, 16023, 38172, 50724, 83...
1          [22656, 25857, 42626, 47747, 19333, 22664, 582...
2          [91841, 59458, 60259, 78503, 67274, 93585, 401...
3                                             [64525, 61175]
4          [2181, 20517, 77915, 104812, 43442, 101367, 51...
                                 ...                        
1371975    [64256, 58501, 59144, 58376, 48776, 59147, 560...
1371976    [29698, 69137, 7192, 52766, 51748, 90151, 4768...
1371977    [1157, 53893, 35721, 78601, 88977, 47892, 1653...
1371978           [89864, 99149, 85872, 95506, 56694, 81751]
1371979                                              [81594]
Name: item_id, Length: 1361243, dtype: object

user_id
0          [16003, 65667, 100484, 16023, 38172, 50724, 83...
1          [22656, 25857, 42626, 47747, 19333, 22664, 582...
2          [91841, 59458, 60259, 78503, 67274, 93585, 401...
3                                             [64525, 61175]
4          [2181, 20517, 77915, 104812, 43442, 101367, 51...
                                 ...                        
1371975    [64256, 58501, 59144, 58376, 48776, 59147, 560...
1371976    [29698, 69137, 7192, 52766, 51748, 90151, 4768...
1371977    [1157, 53893, 35721, 78601, 88977, 47892, 1653...
1371978           [89864, 99149, 85872, 95506, 56694, 81751]
1371979                                              [81594]
Name: item_id, Length: 1361243, dtype: object

AttributeError: 'list' object has no attribute 'str'

In [ ]:
df_recs = uucf(df, '2018-09-20' )